In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pypokerengine.players import BasePokerPlayer
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
import inspect
from pypokerengine.engine.hand_evaluator import HandEvaluator
from pypokerengine.engine.card import Card


In [2]:
class HonestPlayer(BasePokerPlayer):
    def __init__(self, raise_wr = 0.6, call_wr = 0.3, nb_sim=100):
        super().__init__() 
        self.raise_wr = raise_wr
        self.call_wr = call_wr
        self.nb_sim = nb_sim
        
    def declare_action(self, valid_actions, hole_card, round_state):
        curframe = inspect.currentframe()
        calframe = inspect.getouterframes(curframe, 2)
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=self.nb_sim,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= self.raise_wr:
            #print('Raising)', win_rate)
            action = valid_actions[2]  # fetch bet action info
            valid_amounts = action['amount']
            raise_amount =  (valid_amounts['max'] - valid_amounts['min'])/ 5.0 + valid_amounts['min']
            return valid_actions[2]['action'], raise_amount
        elif win_rate >= self.call_wr:
            #print('Calling')
            action = valid_actions[1]  # fetch CALL action info
        else:
            call_amount = valid_actions[1]['amount']
            if(call_amount == 0):
                #print('Calling')
                return valid_actions[1]['action'], valid_actions[1]['amount']
            #print('Folding')
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [3]:
results_dict = {'p1': [], 'p2': [], 'p3': [], 'p4': []}
initial_stack = 1000
for i in range(1):
    config = setup_config(max_round=1000, initial_stack=initial_stack, small_blind_amount=5)
    config.register_player(name="p1", algorithm=HonestPlayer(nb_sim=100))
    config.register_player(name="p2", algorithm=HonestPlayer(raise_wr = 0.9, call_wr=0.7, nb_sim=1000))
    config.register_player(name="p3", algorithm=HonestPlayer(raise_wr = 0.3, call_wr=0.1, nb_sim=10000))
    game_result = start_poker(config, verbose=0)
    #print(i, game_result)
    players = game_result['players']
    for player in players:
        results_dict[player['name']] += [player['stack'] - initial_stack]

print(results_dict)



AttributeError: 'HonestPlayer' object has no attribute '_MessageHandler__parse_ask_message'

In [ ]:

def plot_running_sums(data):
  """Plots the running sums of a dictionary of arrays.

  Args:
    data: A dictionary where the keys are labels and the values are arrays of numbers.
  """

  for label, values in data.items():
    running_sum = np.cumsum(values)
    plt.plot(running_sum, label=label)

  plt.xlabel("Index")
  plt.ylabel("Running Sum")
  plt.legend()
  plt.show()

plot_running_sums(results_dict)